# Debugging Individual Agents in AI Project Analizer

This notebook lets you run and debug each BeeAI agent in isolation. Each section will:
1. Import the necessary agent class.
2. Prepare a minimal environment (e.g., a test ZIP or test files).
3. Call the agent's `handle()` method with simulated events.
4. Capture and display outputs (logs, memory writes, emitted events).

Make sure you have extracted the project into the folder:
```
/mnt/c/blog/ai-project-analizer
```


In [6]:
# Setup paths and logging
import sys, os, shutil, zipfile, tempfile
from pathlib import Path

# Add project src and root to sys.path
sys.path.append(r"/mnt/c/blog/ai-project-analizer/src")
sys.path.append(r"/mnt/c/blog/ai-project-analizer")

# Ensure logs are printed
import logging
logging.basicConfig(level=logging.DEBUG, format="%(asctime)s %(levelname)s %(message)s")

# Create a temporary directory for tests
TEST_DIR = Path("/mnt/c/blog/ai-project-analizer/backup/agent_debug_tests")
if TEST_DIR.exists():
    shutil.rmtree(TEST_DIR)
TEST_DIR.mkdir(parents=True, exist_ok=True)

print("TEST_DIR:", TEST_DIR)


TEST_DIR: /mnt/c/blog/ai-project-analizer/backup/agent_debug_tests


In [16]:
from beeai_framework.agents import BaseAgent as Agent
from beeai_framework.agents.types import Event

ImportError: cannot import name 'Event' from 'beeai_framework.agents.types' (/mnt/c/blog/ai-project-analizer/.venv/lib/python3.12/site-packages/beeai_framework/agents/types.py)

In [7]:
import sys
from pathlib import Path

project_root = Path("/mnt/c/blog/ai-project-analizer")  # adjust as needed
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))


## Create a Sample ZIP for Testing

We'll generate a small ZIP file containing a single `foo.txt`.

In [8]:
# Create a simple zip file with one text file
sample_zip_path = TEST_DIR / "test.zip"
with zipfile.ZipFile(sample_zip_path, "w") as zf:
    # Create a text file inside a temp location
    tmp_file = TEST_DIR / "foo.txt"
    tmp_file.write_text("Hello, world!")
    zf.write(tmp_file, arcname="foo.txt")
print("Created zip at:", sample_zip_path)

Created zip at: /mnt/c/blog/ai-project-analizer/backup/agent_debug_tests/test.zip


In [9]:
from pathlib import Path
import zipfile

# 1) Make sure TEST_DIR and sample_zip_path exist
#    (you presumably already have TEST_DIR from earlier cells).
sample_zip_path = TEST_DIR / "test.zip"

# 2) Create the ZIP _before_ you hand it to the agent:
with zipfile.ZipFile(sample_zip_path, "w") as zf:
    tmp_file = TEST_DIR / "foo.txt"
    tmp_file.write_text("Hello, world!")
    zf.write(tmp_file, arcname="foo.txt")

print("Created zip at:", sample_zip_path)
print("Exists on disk?", sample_zip_path.exists(), "Size:", sample_zip_path.stat().st_size)


Created zip at: /mnt/c/blog/ai-project-analizer/backup/agent_debug_tests/test.zip
Exists on disk? True Size: 125


In [10]:

# 3) Now import and test ZipValidatorAgent
from src.agents.zip_validator_agent import ZipValidatorAgent

zip_agent = ZipValidatorAgent()
emitted = []
zip_agent.emit = lambda event_name, payload: emitted.append((event_name, payload))

# Test with the *existing* ZIP:
event = {"type": "NewUpload", "zip_path": str(sample_zip_path)}
zip_agent.handle(event)
print("Emitted events for valid ZIP:", emitted)

# Test with a truly missing path:
emitted.clear()
event_bad = {"type": "NewUpload", "zip_path": str(TEST_DIR / "nonexistent.zip")}
zip_agent.handle(event_bad)
print("Emitted events for bad ZIP:", emitted)


2025-06-01 10:20:10,500 INFO [config] Loading settings from default environment or .env
2025-06-01 10:20:10,500 DEBUG [config] Final settings: {'BEEAI_MODEL': 'openai/gpt-4o-mini', 'DELETE_TEMP_AFTER_RUN': True, 'ZIP_SIZE_LIMIT_MB': 300, 'MAX_MEMBER_SIZE_MB': 150, 'LOG_LEVEL': 'INFO'}
2025-06-01 10:20:10,501 INFO >>> [zip_validator] handle() entered. event={'type': 'NewUpload', 'zip_path': '/mnt/c/blog/ai-project-analizer/backup/agent_debug_tests/test.zip'}
2025-06-01 10:20:10,501 INFO [zip_validator] Received NewUpload for PosixPath('/mnt/c/blog/ai-project-analizer/backup/agent_debug_tests/test.zip')
2025-06-01 10:20:10,505 DEBUG [zip_validator] Checking size: 125 bytes (max 314572800 bytes)
2025-06-01 10:20:10,505 DEBUG [zip_validator] Checking if PosixPath('/mnt/c/blog/ai-project-analizer/backup/agent_debug_tests/test.zip') is a zipfile
2025-06-01 10:20:10,507 DEBUG [zip_validator] Performing CRC check on PosixPath('/mnt/c/blog/ai-project-analizer/backup/agent_debug_tests/test.zip')

Emitted events for valid ZIP: [('ZipValid', {'zip_path': '/mnt/c/blog/ai-project-analizer/backup/agent_debug_tests/test.zip'})]
Emitted events for bad ZIP: [('ZipInvalid', {'zip_path': '/mnt/c/blog/ai-project-analizer/backup/agent_debug_tests/nonexistent.zip', 'reason': 'File does not exist'})]


In [11]:
from src.agents.zip_validator_agent import ZipValidatorAgent

# Prepare the agent
zip_agent = ZipValidatorAgent()
emitted = []
# Monkey‐patch emit to capture events
zip_agent.emit = lambda event_name, payload: emitted.append((event_name, payload))

# Test with valid ZIP (assuming sample_zip_path is defined)
event = {"type": "NewUpload", "zip_path": str(sample_zip_path)}
zip_agent.handle(event)
print("Emitted events for valid ZIP:", emitted)

# Test with an invalid path
emitted.clear()
event_bad = {"type": "NewUpload", "zip_path": str(TEST_DIR / "nonexistent.zip")}
zip_agent.handle(event_bad)
print("Emitted events for bad ZIP:", emitted)


2025-06-01 10:20:15,957 INFO >>> [zip_validator] handle() entered. event={'type': 'NewUpload', 'zip_path': '/mnt/c/blog/ai-project-analizer/backup/agent_debug_tests/test.zip'}
2025-06-01 10:20:15,957 INFO [zip_validator] Received NewUpload for PosixPath('/mnt/c/blog/ai-project-analizer/backup/agent_debug_tests/test.zip')
2025-06-01 10:20:15,962 DEBUG [zip_validator] Checking size: 125 bytes (max 314572800 bytes)
2025-06-01 10:20:15,963 DEBUG [zip_validator] Checking if PosixPath('/mnt/c/blog/ai-project-analizer/backup/agent_debug_tests/test.zip') is a zipfile
2025-06-01 10:20:15,966 DEBUG [zip_validator] Performing CRC check on PosixPath('/mnt/c/blog/ai-project-analizer/backup/agent_debug_tests/test.zip')
2025-06-01 10:20:15,969 INFO [zip_validator] ZIP archive PosixPath('/mnt/c/blog/ai-project-analizer/backup/agent_debug_tests/test.zip') passed all checks
2025-06-01 10:20:15,969 INFO [zip_validator] Emitted ZipValid for PosixPath('/mnt/c/blog/ai-project-analizer/backup/agent_debug_tes

Emitted events for valid ZIP: [('ZipValid', {'zip_path': '/mnt/c/blog/ai-project-analizer/backup/agent_debug_tests/test.zip'})]
Emitted events for bad ZIP: [('ZipInvalid', {'zip_path': '/mnt/c/blog/ai-project-analizer/backup/agent_debug_tests/nonexistent.zip', 'reason': 'File does not exist'})]


## Debug `ZipValidatorAgent`

Instantiate `ZipValidatorAgent`, monkey-patch its `emit` to capture events, and run with our test ZIP.

In [14]:
from src.agents.zip_validator_agent import ZipValidatorAgent

# Prepare the agent
zip_agent = ZipValidatorAgent()
emitted = []
# Monkey-patch emit to capture events
zip_agent.emit = lambda event_name, payload: emitted.append((event_name, payload))

# Test with valid ZIP
event = {"type": "NewUpload", "zip_path": str(sample_zip_path)}
zip_agent.handle(event)
print("Emitted events for valid ZIP:", emitted)

# Test with an invalid path
emitted.clear()
event_bad = {"type": "NewUpload", "zip_path": str(TEST_DIR / "nonexistent.zip")}
zip_agent.handle(event_bad)
print("Emitted events for bad ZIP:", emitted)

ModuleNotFoundError: No module named 'beeai_framework.agent'

## Debug `ExtractionAgent`

Use the valid ZIP event and capture `FileDiscovered` and `ExtractionDone`.

In [22]:
from src.agents.extraction_agent import ExtractionAgent

# Prepare the agent
ext_agent = ExtractionAgent()
emitted_ext = []
ext_agent.emit = lambda event_name, payload: emitted_ext.append((event_name, payload))

# Run extraction on the valid ZIP event
event_zip_valid = {"type": "ZipValid", "zip_path": str(sample_zip_path)}
ext_agent.handle(event_zip_valid)
print("Extraction emitted events:")
for evt in emitted_ext:
    print(evt)

# Check extracted directory
if emitted_ext and emitted_ext[-1][0] == "ExtractionDone":
    base_dir = Path(emitted_ext[-1][1]["base_dir"])
    print("Contents of extracted directory:", list(base_dir.rglob("*")))


ModuleNotFoundError: No module named 'beeai_framework.agent'

In [12]:
from pathlib import Path
import zipfile

# (1) Create the test ZIP if you haven't already)
sample_zip_path = TEST_DIR / "test.zip"
with zipfile.ZipFile(sample_zip_path, "w") as zf:
    tmp = TEST_DIR / "foo.txt"
    tmp.write_text("Hello from extraction test!")
    zf.write(tmp, arcname="foo.txt")

print("ZIP exists: ", sample_zip_path.exists())

# (2) Now import and test ExtractionAgent:
from src.agents.extraction_agent import ExtractionAgent

ext_agent = ExtractionAgent()
emitted_ext = []
# Monkey-patch emit to capture events
ext_agent.emit = lambda event_name, payload: emitted_ext.append((event_name, payload))

# (3) Send the ZipValid event:
event_zip_valid = {"type": "ZipValid", "zip_path": str(sample_zip_path)}
ext_agent.handle(event_zip_valid)

print("\nExtraction emitted events:")
for evt in emitted_ext:
    print(evt)

# (4) If extraction succeeded, the last event should be ExtractionDone:
if emitted_ext and emitted_ext[-1][0] == "ExtractionDone":
    base_dir = Path(emitted_ext[-1][1]["base_dir"])
    print("\nContents of extracted directory:")
    for p in sorted(base_dir.rglob("*")):
        print("  ", p)
else:
    print("\nNo ExtractionDone event; maybe ExtractionFailed was emitted above.")


2025-06-01 10:20:23,409 INFO >>> [extraction] handle() entered. event={'type': 'ZipValid', 'zip_path': '/mnt/c/blog/ai-project-analizer/backup/agent_debug_tests/test.zip'}
2025-06-01 10:20:23,409 INFO [extraction] Received ZipValid for PosixPath('/mnt/c/blog/ai-project-analizer/backup/agent_debug_tests/test.zip')
2025-06-01 10:20:23,409 DEBUG [extraction] Calling safe_extract for PosixPath('/mnt/c/blog/ai-project-analizer/backup/agent_debug_tests/test.zip')
2025-06-01 10:20:23,410 INFO [extraction] safe_extract() called with PosixPath('/mnt/c/blog/ai-project-analizer/backup/agent_debug_tests/test.zip')
2025-06-01 10:20:23,410 INFO [extraction] Created temporary directory PosixPath('/tmp/ai_analyser_4eae4w6d')
2025-06-01 10:20:23,412 DEBUG [extraction] Inspecting member 'foo.txt' (size=27 bytes)
2025-06-01 10:20:23,412 INFO [extraction] All members passed size and Zip‐Slip checks, extracting now...
2025-06-01 10:20:23,413 INFO [extraction] Extraction completed into PosixPath('/tmp/ai_an

ZIP exists:  True

Extraction emitted events:
('FileDiscovered', {'path': '/tmp/ai_analyser_4eae4w6d/foo.txt'})
('ExtractionDone', {'base_dir': '/tmp/ai_analyser_4eae4w6d'})

Contents of extracted directory:
   /tmp/ai_analyser_4eae4w6d/foo.txt


## Debug `TreeBuilderAgent`

Simulate `FileDiscovered` events, then `ExtractionDone` with the previously extracted folder.

In [24]:
from src.agents.tree_builder_agent import TreeBuilderAgent

# Find the base_dir from previous extraction
if emitted_ext and emitted_ext[-1][0] == "ExtractionDone":
    base_dir = Path(emitted_ext[-1][1]["base_dir"])
else:
    raise RuntimeError("Extraction did not complete successfully.")

# Prepare tree builder
tree_agent = TreeBuilderAgent()
# Send FileDiscovered events (for each file in base_dir)
for file_path in base_dir.rglob("*"):
    if file_path.is_file():
        file_event = {"type": "FileDiscovered", "path": str(file_path)}
        tree_agent.handle(file_event)

# Now fire ExtractionDone
done_event = {"type": "ExtractionDone", "base_dir": str(base_dir)}
tree_agent.handle(done_event)

# The tree text is stored in memory
print("Generated project_tree.txt:")
print(tree_agent.memory.get("project_tree.txt", "<none>"))


ModuleNotFoundError: No module named 'beeai_framework.agent'

In [13]:
from pathlib import Path

# (1) Ensure “emitted_ext” contains an ExtractionDone event, e.g.:
#     emitted_ext = [('FileDiscovered', {...}), ('ExtractionDone', {'base_dir': '/tmp/…'})]

if emitted_ext and emitted_ext[-1][0] == "ExtractionDone":
    base_dir = Path(emitted_ext[-1][1]["base_dir"])
else:
    raise RuntimeError("Extraction did not complete successfully.")

# (2) Prepare TreeBuilderAgent and capture its emits
from src.agents.tree_builder_agent import TreeBuilderAgent

tree_agent = TreeBuilderAgent()
emitted_tree = []
tree_agent.emit = lambda event_name, payload: emitted_tree.append((event_name, payload))

# (3) Send FileDiscovered events for each file under base_dir
for file_path in base_dir.rglob("*"):
    if file_path.is_file():
        file_event = {"type": "FileDiscovered", "path": str(file_path)}
        tree_agent.handle(file_event)

# (4) Fire ExtractionDone
done_event = {"type": "ExtractionDone", "base_dir": str(base_dir)}
tree_agent.handle(done_event)

# (5) Check emitted events and memory
print("Emitted events from TreeBuilderAgent:")
print(emitted_tree)  # Expect [('TreeBuilt', {'tree_path': 'project_tree.txt'})]

print("\nGenerated project_tree.txt in memory:")
print(tree_agent.memory.get("project_tree.txt", "<none>"))


2025-06-01 10:20:28,542 INFO [tree_builder] Initialized with empty paths list and no base_dir
2025-06-01 10:20:28,543 INFO >>> [tree_builder] handle() entered. event={'type': 'FileDiscovered', 'path': '/tmp/ai_analyser_4eae4w6d/foo.txt'}
2025-06-01 10:20:28,543 DEBUG [tree_builder] Received FileDiscovered for PosixPath('/tmp/ai_analyser_4eae4w6d/foo.txt')
2025-06-01 10:20:28,544 INFO [tree_builder] Added PosixPath('/tmp/ai_analyser_4eae4w6d/foo.txt') to paths (total now: 1)
2025-06-01 10:20:28,544 INFO >>> [tree_builder] handle() entered. event={'type': 'ExtractionDone', 'base_dir': '/tmp/ai_analyser_4eae4w6d'}
2025-06-01 10:20:28,544 INFO [tree_builder] Received ExtractionDone, base_dir set to PosixPath('/tmp/ai_analyser_4eae4w6d')
2025-06-01 10:20:28,545 INFO [tree_builder] _build_and_emit_tree() called with base_dir=PosixPath('/tmp/ai_analyser_4eae4w6d') and 1 paths
2025-06-01 10:20:28,545 DEBUG [tree_builder] Using rich to build tree
2025-06-01 10:20:28,546 DEBUG [tree_builder] Add

ai_analyser_4eae4w6d/
└── foo.txt

2025-06-01 10:20:28,548 INFO [tree_builder] Rich tree built successfully (length=34 chars)
2025-06-01 10:20:28,548 INFO [tree_builder] Stored 'project_tree.txt' in memory (length=34)
2025-06-01 10:20:28,549 INFO [tree_builder] Emitted TreeBuilt with path 'project_tree.txt'


Emitted events from TreeBuilderAgent:
[('TreeBuilt', {'tree_path': 'project_tree.txt'})]

Generated project_tree.txt in memory:
ai_analyser_4eae4w6d/
└── foo.txt



## Debug `FileTriageAgent`

Queue discovered files, flush on `ExtractionDone`, and capture `FileForAnalysis` and `TriageComplete`.

In [26]:
from src.agents.file_triage_agent import FileTriageAgent

# Prepare file triage agent
triage_agent = FileTriageAgent()
emitted_triage = []
triage_agent.emit = lambda event_name, payload: emitted_triage.append((event_name, payload))

# Send FileDiscovered events for all files under base_dir
for file_path in base_dir.rglob("*"):
    if file_path.is_file():
        file_event = {"type": "FileDiscovered", "path": str(file_path)}
        triage_agent.handle(file_event)

# Now send ExtractionDone
done_event = {"type": "ExtractionDone", "base_dir": str(base_dir)}
triage_agent.handle(done_event)

print("Triage emitted events:")
for evt in emitted_triage:
    print(evt)


ModuleNotFoundError: No module named 'beeai_framework.agent'

In [14]:
from pathlib import Path

# (1) Make sure `base_dir` came from ExtractionDone; e.g.:
#     emitted_ext = [('FileDiscovered', {...}), ('ExtractionDone', {'base_dir': '/tmp/…'})]

if emitted_ext and emitted_ext[-1][0] == "ExtractionDone":
    base_dir = Path(emitted_ext[-1][1]["base_dir"])
else:
    raise RuntimeError("Extraction did not complete successfully.")

# (2) Prepare FileTriageAgent and capture emits
from src.agents.file_triage_agent import FileTriageAgent

triage_agent = FileTriageAgent()
emitted_triage = []
triage_agent.emit = lambda event_name, payload: emitted_triage.append((event_name, payload))

# (3) Send FileDiscovered for each file under base_dir
for file_path in base_dir.rglob("*"):
    if file_path.is_file():
        file_event = {"type": "FileDiscovered", "path": str(file_path)}
        triage_agent.handle(file_event)

# (4) Send ExtractionDone
done_event = {"type": "ExtractionDone", "base_dir": str(base_dir)}
triage_agent.handle(done_event)

# (5) Inspect what was emitted
print("Triage emitted events:")
for evt in emitted_triage:
    print(evt)


2025-06-01 10:20:36,957 INFO [file_triage] Initialized with empty queue and extraction_done=False
2025-06-01 10:20:36,958 INFO >>> [file_triage] handle() entered. event={'type': 'FileDiscovered', 'path': '/tmp/ai_analyser_4eae4w6d/foo.txt'}
2025-06-01 10:20:36,959 DEBUG [file_triage] Received FileDiscovered for PosixPath('/tmp/ai_analyser_4eae4w6d/foo.txt')
2025-06-01 10:20:36,959 DEBUG [file_triage] _handle_file() called with PosixPath('/tmp/ai_analyser_4eae4w6d/foo.txt')
2025-06-01 10:20:36,960 DEBUG [file_io_tool] looks_binary() called for PosixPath('/tmp/ai_analyser_4eae4w6d/foo.txt') (sample=1024 bytes)
2025-06-01 10:20:36,960 DEBUG [file_io_tool] File PosixPath('/tmp/ai_analyser_4eae4w6d/foo.txt'): non_printable=0, total=27, ratio=0.00
2025-06-01 10:20:36,961 INFO [file_io_tool] looks_binary() returns False for PosixPath('/tmp/ai_analyser_4eae4w6d/foo.txt')
2025-06-01 10:20:36,961 DEBUG [file_io_tool] priority_score() called for PosixPath('/tmp/ai_analyser_4eae4w6d/foo.txt')
2025

Triage emitted events:
('FileForAnalysis', {'path': '/tmp/ai_analyser_4eae4w6d/foo.txt', 'score': 70})
('TriageComplete', {})


## Debug `FileAnalysisAgent`

Analyze each file from `FileForAnalysis` events and finalize on `TriageComplete`. Display `file_summaries.json`.

In [15]:
from src.agents.file_analysis_agent import FileAnalysisAgent
import json

# Prepare file analysis agent
analysis_agent = FileAnalysisAgent()
emitted_analysis = []
analysis_agent.emit = lambda event_name, payload: emitted_analysis.append((event_name, payload))

# Simulate FileForAnalysis events from triage
file_for_analysis_events = [evt for evt in emitted_triage if evt[0] == "FileForAnalysis"]
for _, payload in file_for_analysis_events:
    evt = {"type": "FileForAnalysis", "path": payload["path"], "score": payload.get("score", 0)}
    analysis_agent.handle(evt)

# Now fire TriageComplete
tc_event = {"type": "TriageComplete"}
analysis_agent.handle(tc_event)

print("FileAnalysis emitted events:")
for evt in emitted_analysis:
    print(evt)

# Check memory for file_summaries.json
print("Memory file_summaries.json:")
print(analysis_agent.memory.get("file_summaries.json", "<none>"))


ModuleNotFoundError: No module named 'beeai_framework.agent'

In [16]:
from src.agents.file_analysis_agent import FileAnalysisAgent
import json

# (1) Prepare the agent and capture emits
analysis_agent = FileAnalysisAgent()
emitted_analysis = []
analysis_agent.emit = lambda event_name, payload: emitted_analysis.append((event_name, payload))

# (2) Simulate FileForAnalysis events from triage
file_for_analysis_events = [evt for evt in emitted_triage if evt[0] == "FileForAnalysis"]
for _, payload in file_for_analysis_events:
    evt = {"type": "FileForAnalysis", "path": payload["path"], "score": payload.get("score", 0)}
    analysis_agent.handle(evt)

# (3) Simulate ExtractionDone (so that base_dir is set if needed)
#     If you want the agent to know base_dir, do:
if emitted_ext and emitted_ext[-1][0] == "ExtractionDone":
    base_dir_evt = {"type": "ExtractionDone", "base_dir": emitted_ext[-1][1]["base_dir"]}
    analysis_agent.handle(base_dir_evt)

# (4) Now fire TriageComplete
tc_event = {"type": "TriageComplete"}
analysis_agent.handle(tc_event)

# (5) Inspect emitted events and memory
print("FileAnalysis emitted events:")
for evt in emitted_analysis:
    print(evt)

print("\nMemory file_summaries.json:")
print(analysis_agent.memory.get("file_summaries.json", "<none>"))


2025-06-01 10:22:59,488 INFO >>> [file_analysis] handle() entered. event={'type': 'FileForAnalysis', 'path': '/tmp/ai_analyser_4eae4w6d/foo.txt', 'score': 70}
2025-06-01 10:22:59,488 INFO [file_analysis] Received FileForAnalysis for PosixPath('/tmp/ai_analyser_4eae4w6d/foo.txt')
2025-06-01 10:22:59,488 DEBUG [file_analysis] _analyse() called with path=PosixPath('/tmp/ai_analyser_4eae4w6d/foo.txt'), base=PosixPath('/tmp/ai_analyser_4eae4w6d')
2025-06-01 10:22:59,489 DEBUG [file_analysis] analyse_file() called for PosixPath('/tmp/ai_analyser_4eae4w6d/foo.txt'), base=PosixPath('/tmp/ai_analyser_4eae4w6d')
2025-06-01 10:22:59,489 DEBUG [file_analysis] Read 27 characters from PosixPath('/tmp/ai_analyser_4eae4w6d/foo.txt')
2025-06-01 10:22:59,490 INFO [file_analysis] Fallback text summary for PosixPath('/tmp/ai_analyser_4eae4w6d/foo.txt'): 'Hello from extraction test!'
2025-06-01 10:22:59,490 INFO [file_analysis] Analysis result for PosixPath('/tmp/ai_analyser_4eae4w6d/foo.txt'): {'rel_path'

FileAnalysis emitted events:
('FileAnalysed', {'rel_path': 'foo.txt', 'kind': 'text', 'summary': 'Hello from extraction test!'})
('AnalysisComplete', {})

Memory file_summaries.json:
[
  {
    "rel_path": "foo.txt",
    "kind": "text",
    "summary": "Hello from extraction test!"
  }
]


## Debug `SummarySynthesizerAgent`

Feed `TreeBuilt` and `FileAnalysed` / `AnalysisComplete` events, then inspect `project_summary.txt`.

In [17]:
from src.agents.summary_synthesizer_agent import SummarySynthesizerAgent

# Prepare summary synthesizer agent
summary_agent = SummarySynthesizerAgent()
emitted_summary = []
summary_agent.emit = lambda event_name, payload: emitted_summary.append((event_name, payload))

# Manually populate memory for tree text and file summaries
summary_agent.memory["project_tree.txt"] = tree_agent.memory.get("project_tree.txt", "")
# Use the list of analyses from emitted_analysis if any; else simulate one
analyses = [payload for (name, payload) in emitted_analysis if name == "FileAnalysed"]
for analysis in analyses:
    evt = {"type": "FileAnalysed", **analysis}
    summary_agent.handle(evt)

# Now signal AnalysisComplete
ac_event = {"type": "AnalysisComplete"}
summary_agent.handle(ac_event)

# Also signal TreeBuilt (to ensure it has the tree)
tb_event = {"type": "TreeBuilt", "tree_path": "project_tree.txt"}
summary_agent.handle(tb_event)

print("SummarySynthesizer emitted events:")
for evt in emitted_summary:
    print(evt)

# Check memory for project_summary.txt
print("\nGenerated project_summary.txt:")
print(summary_agent.memory.get("project_summary.txt", "<none>"))

ModuleNotFoundError: No module named 'beeai_framework.agent'

In [18]:
from src.agents.summary_synthesizer_agent import SummarySynthesizerAgent

# Prepare summary synthesizer agent
summary_agent = SummarySynthesizerAgent()
emitted_summary = []
summary_agent.emit = lambda event_name, payload: emitted_summary.append((event_name, payload))

# (1) Populate memory with the tree text from TreeBuilderAgent
summary_agent.memory["project_tree.txt"] = tree_agent.memory.get("project_tree.txt", "")

# (2) Feed in all FileAnalysed events (payloads) from file_analysis_agent
analyses = [payload for (name, payload) in emitted_analysis if name == "FileAnalysed"]
for analysis in analyses:
    evt = {"type": "FileAnalysed", **analysis}
    summary_agent.handle(evt)

# (3) Signal that analysis is complete
ac_event = {"type": "AnalysisComplete"}
summary_agent.handle(ac_event)

# (4) Also signal that the tree was built
tb_event = {"type": "TreeBuilt", "tree_path": "project_tree.txt"}
summary_agent.handle(tb_event)

# (5) Inspect what got emitted
print("SummarySynthesizer emitted events:")
for evt in emitted_summary:
    print(evt)

# (6) Check memory for the final project_summary.txt
print("\nGenerated project_summary.txt:")
print(summary_agent.memory.get("project_summary.txt", "<none>"))


2025-06-01 10:25:18,813 INFO [summary_synthesizer] Initialized with model='openai/gpt-4o-mini'
2025-06-01 10:25:18,813 INFO >>> [summary_synthesizer] handle() entered. event={'type': 'FileAnalysed', 'rel_path': 'foo.txt', 'kind': 'text', 'summary': 'Hello from extraction test!'}
2025-06-01 10:25:18,814 DEBUG [summary_synthesizer] Received FileAnalysed: {'type': 'FileAnalysed', 'rel_path': 'foo.txt', 'kind': 'text', 'summary': 'Hello from extraction test!'}
2025-06-01 10:25:18,814 INFO [summary_synthesizer] Appended analysis for 'foo.txt', total=1
2025-06-01 10:25:18,814 INFO >>> [summary_synthesizer] handle() entered. event={'type': 'AnalysisComplete'}
2025-06-01 10:25:18,814 INFO [summary_synthesizer] Received AnalysisComplete event
2025-06-01 10:25:18,815 INFO [summary_synthesizer] _maybe_finish() called. analysis_done=True, tree_text_set=False
2025-06-01 10:25:18,815 DEBUG [summary_synthesizer] Waiting for both tree and analyses
2025-06-01 10:25:18,815 INFO >>> [summary_synthesizer]

SummarySynthesizer emitted events:
('ProjectDraft', {'draft': 'The dominant file type is *text* (count: 1). Inferred tech stack: Unknown or mixed‐language project.'})
('SummaryPolished', {'summary_path': 'project_summary.txt'})

Generated project_summary.txt:
The dominant file type is *text* (count: 1). Inferred tech stack: Unknown or mixed‐language project.
